In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import SimpleITK as sitk

import sys
import os

<h3>Intensity-Based Registration</h3>

In [14]:
# load fixed and moving images to register
fixed = sitk.ReadImage("LIDC_IDRI_axial_01.nii", sitk.sitkFloat32)
moving = sitk.ReadImage("2.000000-ROUTINE CHEST NON-CON-50599.nii", sitk.sitkFloat32)

In [15]:
def command_iteration(method):
    print(f"{method.GetOptimizerIteration():3} = {method.GetMetricValue():10.5f} : {method.GetOptimizerPosition()}")

R = sitk.ImageRegistrationMethod()
R.SetMetricAsMeanSquares()
R.SetOptimizerAsRegularStepGradientDescent(4.0, .01, 200)
R.SetInitialTransform(sitk.TranslationTransform(fixed.GetDimension()))
R.SetInterpolator(sitk.sitkLinear)

R.AddCommand(sitk.sitkIterationEvent, lambda: command_iteration(R))

outTx = R.Execute(fixed, moving)

print("-------")
print(outTx)
print(f"Optimizer stop condition: {R.GetOptimizerStopConditionDescription()}")
print(f" Iteration: {R.GetOptimizerIteration()}")
print(f" Metric value: {R.GetMetricValue()}")

sitk.WriteTransform(outTx, "registered_intensity_3D.tfm")

if ("SITK_NOSHOW" not in os.environ):
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(fixed)
    resampler.SetInterpolator(sitk.sitkLinear)
    resampler.SetDefaultPixelValue(100)
    resampler.SetTransform(outTx)

    out = resampler.Execute(moving)
    simg1 = sitk.Cast(sitk.RescaleIntensity(fixed), sitk.sitkUInt8)
    simg2 = sitk.Cast(sitk.RescaleIntensity(out), sitk.sitkUInt8)
    cimg = sitk.Compose(simg1, simg2, simg1 // 2. + simg2 // 2.)
    sitk.Show(cimg, "ImageRegistration1 Composition")

  0 = 524504.16406 : (-3.255615129775106, 2.323851908040856, -0.02612348128207581)
  1 = 489938.05798 : (-6.542470298055147, 4.603314059596745, -0.05133067023662149)
  2 = 452205.97909 : (-9.79413712880736, 6.932803701419525, -0.06319786983137111)
  3 = 416939.07293 : (-12.76161411445948, 9.61496881791417, -0.05479385386992799)
  4 = 402008.21773 : (-10.985549997729429, 8.695667364927838, -0.032860048884778496)
  5 = 399418.20032 : (-11.958846060734587, 8.924893475842117, -0.020605914326384638)
  6 = 392508.13118 : (-11.459455484396903, 8.903660965110012, -0.00802685422126979)
  7 = 391997.58840 : (-11.709047851419925, 8.904601948379502, 0.0062126989434687484)
  8 = 393637.68318 : (-11.584470001611326, 8.907717159856604, 0.015993025856166077)
  9 = 392361.30888 : (-11.641178011422403, 8.904091815816395, 0.04201817722665635)
 10 = 392964.07601 : (-11.610650811742476, 8.906043910869883, 0.04840894436402031)
 11 = 392639.32004 : (-11.588279340827354, 8.90756112767456, 0.07017538280170318)

<h3>Intensity-Based Registration 2</h3>

In [3]:
# load fixed and moving images to register
fixed = sitk.ReadImage("LIDC_IDRI_axial_01.nii", sitk.sitkFloat32)
moving = sitk.ReadImage("2.000000-ROUTINE CHEST NON-CON-50599.nii", sitk.sitkFloat32)

In [6]:
def command_iteration(method):
    print(f"{method.GetOptimizerIteration():3} = {method.GetMetricValue():7.5f} : {method.GetOptimizerPosition()}")
    
pixelType = sitk.sitkFloat32

# fixed = sitk.ReadImage(fixed, sitk.sitkFloat32)
fixed = sitk.Normalize(fixed)
fixed = sitk.DiscreteGaussian(fixed, 2.0)

# moving = sitk.ReadImage(moving, sitk.sitkFloat32)
moving = sitk.Normalize(moving)
moving = sitk.DiscreteGaussian(moving, 2.0)

R = sitk.ImageRegistrationMethod()

R.SetMetricAsJointHistogramMutualInformation()

R.SetOptimizerAsGradientDescentLineSearch(learningRate=1.0,
                                          numberOfIterations=200,
                                          convergenceMinimumValue=1e-5,
                                          convergenceWindowSize=5)

R.SetInitialTransform(sitk.TranslationTransform(fixed.GetDimension()))

R.SetInterpolator(sitk.sitkLinear)

R.AddCommand(sitk.sitkIterationEvent, lambda: command_iteration(R))

outTx = R.Execute(fixed, moving)

  0 = -6.00525 : (-2.19289780372709e-05, 0.00019392330682359204, 9.511879942640494e-05)
  1 = -6.00525 : (-2.2107751643959872e-05, 0.00019550397928203058, 9.589411880297467e-05)
  2 = -6.00525 : (-2.2218068648539625e-05, 0.00019647937198468129, 9.637254847868369e-05)
  3 = -6.00525 : (-2.2328198981061636e-05, 0.0001974531136269462, 9.685016824670252e-05)
  4 = -6.00525 : (-2.2541899714930186e-05, 0.00019934259441025737, 9.777695746127515e-05)


In [11]:
print("-------")
print(outTx)
print(f"Optimizer stop condition: {R.GetOptimizerStopConditionDescription()}")
print(f" Iteration: {R.GetOptimizerIteration()}")
print(f" Metric value: {R.GetMetricValue()}")

sitk.WriteTransform(outTx, "LIDC_IDRI_axial_02.tfm")

if ("SITK_NOSHOW" not in os.environ):
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(fixed)
    resampler.SetInterpolator(sitk.sitkLinear)
    resampler.SetDefaultPixelValue(1)
    resampler.SetTransform(outTx)

    out = resampler.Execute(moving)

    simg1 = sitk.Cast(sitk.RescaleIntensity(fixed), sitk.sitkUInt8)
    simg2 = sitk.Cast(sitk.RescaleIntensity(out), sitk.sitkUInt8)
    cimg = sitk.Compose(simg1, simg2, simg1 // 2. + simg2 // 2.)
    sitk.Show(cimg, "Image Registration")

-------
itk::simple::Transform
 TranslationTransform (0x7f9f00f32e00)
   RTTI typeinfo:   itk::TranslationTransform<double, 3u>
   Reference Count: 2
   Modified Time: 19806
   Debug: Off
   Object Name: 
   Observers: 
     none
   Offset: [-2.1929e-05, 0.000193923, 9.51188e-05]

Optimizer stop condition: GradientDescentLineSearchOptimizerv4Template: Convergence checker passed at iteration 5.
 Iteration: 5
 Metric value: -6.005254598240413


<h3>Similarity 3D Transformation</h3>

In [2]:
# dicom_dir = "./LIDC-IDRI/manifest-1600709154662/LIDC-IDRI/LIDC-IDRI-0001/01-01-2000-NA-NA-30178/3000566.000000-NA-03192/"

# print("Reading Dicom directory:", dicom_dir)
# reader = sitk.ImageSeriesReader()

# dicom_names = reader.GetGDCMSeriesFileNames(dicom_dir)
# reader.SetFileNames(dicom_names)

# image = reader.Execute()

# size = image.GetSize()
# print("Image size:", size[0], size[1], size[2])

# print("Writing image:", "LIDC_IDRI_axial_01.nii")

# sitk.WriteImage(image, "LIDC_IDRI_axial_01.nii")

# if ("SITK_NOSHOW" not in os.environ):
#     sitk.Show(image, "Dicom Series")

In [3]:
# load fixed and moving images to register
fixed = sitk.ReadImage("LIDC_IDRI_axial_01.nii", sitk.sitkFloat32)
moving = sitk.ReadImage("2.000000-ROUTINE CHEST NON-CON-50599.nii", sitk.sitkFloat32)

In [4]:
R = sitk.ImageRegistrationMethod()

R.SetMetricAsCorrelation()

R.SetOptimizerAsRegularStepGradientDescent(learningRate=2.0,
                                           minStep=1e-4,
                                           numberOfIterations=500,
                                           gradientMagnitudeTolerance=1e-8)
R.SetOptimizerScalesFromIndexShift()

tx = sitk.CenteredTransformInitializer(fixed, moving, sitk.Similarity3DTransform())

In [5]:
def command_iteration(method):
    if (method.GetOptimizerIteration() == 0):
        print("Estimated Scales: ", method.GetOptimizerScales())
    print(f"{method.GetOptimizerIteration():3} = {method.GetMetricValue():7.5f} : {method.GetOptimizerPosition()}")

R.SetInitialTransform(tx)

R.SetInterpolator(sitk.sitkLinear)

R.AddCommand(sitk.sitkIterationEvent, lambda: command_iteration(R))

outTx = R.Execute(fixed, moving)

Estimated Scales:  (77766.81562058833, 77150.3963728492, 135162.23070764923, 2.0227160493790284, 2.0227160493905236, 0.6399999999987221, 152999.353016389)
  0 = -0.74770 : (-0.00019472214608236242, 0.0007758282028360315, -0.00017238707743845168, -10.664098433911642, 9.296493923421538, 5.276598119545042, 1.0043824363873257)
  1 = -0.74758 : (-0.00034569158220680487, 0.0010706838821493009, -0.00012854325238347166, -11.532498289729325, 8.808882351632258, 5.186512325603588, 1.0035172272329715)
  2 = -0.75363 : (-0.0007576793537345701, 0.0017871418883573138, -0.0003395423541208101, -11.904358123481611, 8.470578119196285, 4.32209737862947, 0.9961416233972097)
  3 = -0.74785 : (-0.0007475255563833205, 0.0015940725058443848, -0.00036630216528021935, -11.64018557510421, 8.89017219030324, 4.257858774437339, 1.0013309901035186)
  4 = -0.76019 : (-0.000265957832027513, 0.0004431785000307785, -0.0004495765795756355, -11.349610186086563, 8.73972443389697, 3.879848160897685, 1.0070875807631907)
  5 =

 48 = -0.75778 : (-0.0004868686032902062, 0.0011664687402186654, -0.0012532667203823751, -11.615568008765546, 8.802382254251158, 0.759085931945206, 1.0026428160810406)
 49 = -0.75767 : (-0.0004832271077744338, 0.001175455290332879, -0.0012688524307894944, -11.561271752368771, 8.797630329572428, 0.7284986340562106, 1.0026026506955332)
 50 = -0.75778 : (-0.0004852657935296395, 0.0011699977355536166, -0.0012717438088894204, -11.588809725798589, 8.799957108130165, 0.7139106931176491, 1.0026234085890045)
 51 = -0.75773 : (-0.0004860449300397669, 0.0011695621004522224, -0.00128311219039318, -11.589643710102413, 8.799945191147444, 0.6826718341784093, 1.00262384702503)
 52 = -0.75773 : (-0.00048767053934405336, 0.0011703911648296011, -0.001294748740201535, -11.588723837390399, 8.800044039898058, 0.6514355410561791, 1.002624224128653)
 53 = -0.75772 : (-0.0004895379383649946, 0.001170492968011019, -0.0013066425234779596, -11.590065324137871, 8.7999242124903, 0.6202145869813884, 1.00262545657944

 97 = -0.75775 : (-0.0004989197763180445, 0.0011965096162494487, -0.0016300002677508698, -11.599177799589414, 8.797477721463704, -0.1587409823130053, 1.002635698000005)
 98 = -0.75773 : (-0.0004983998717382307, 0.0011984103781870156, -0.0016327060672194271, -11.592362650260778, 8.796753308444657, -0.16249118801537948, 1.002629864668669)
 99 = -0.75774 : (-0.0004983137106766896, 0.0011984156712297918, -0.001637108376150532, -11.592658962636847, 8.796768419581214, -0.17029804713476024, 1.0026299994798002)
100 = -0.75774 : (-0.000498004463313925, 0.0011988384940487932, -0.001641606487124563, -11.592447003059759, 8.79678740868588, -0.17810764294786616, 1.002629912287873)
101 = -0.75775 : (-0.0004974324775871359, 0.0011993982721036826, -0.001646108816575869, -11.592912570091235, 8.796881294494197, -0.18590568790455297, 1.002630413775688)
102 = -0.75775 : (-0.0004969160061605184, 0.001200038374744689, -0.0016507684801045487, -11.59244388431882, 8.796802064044178, -0.19370370833887857, 1.0026

146 = -0.75779 : (-0.000503764406265655, 0.0011992235628643527, -0.0017849870705520599, -11.591582890538003, 8.795211899460663, -0.3639597210047606, 1.0026314992831915)
147 = -0.75780 : (-0.000503927610898653, 0.001198761231488521, -0.0017858832969957022, -11.59329318940628, 8.795374879607984, -0.36488870228202025, 1.0026329588849636)
148 = -0.75780 : (-0.0005040083124650311, 0.0011987947345422894, -0.00178809751665127, -11.593376749049217, 8.795370000700007, -0.3668400278799088, 1.0026330904080027)
149 = -0.75780 : (-0.0005041229613892657, 0.0011987875933357726, -0.0017903577051950619, -11.59330300884725, 8.795323873407257, -0.3687912099407149, 1.0026330962143601)
150 = -0.75780 : (-0.0005042049467715037, 0.001198765526570132, -0.0017926329898154932, -11.593427340271846, 8.795332118404742, -0.37074035082995843, 1.0026332957057784)
151 = -0.75780 : (-0.0005042816382366154, 0.00119869024883515, -0.001794978325794345, -11.593271876088345, 8.795277296381025, -0.3726865010378915, 1.0026331

195 = -0.75787 : (-0.0005053315367481499, 0.0011984388826697132, -0.0018688771056849386, -11.593857805596373, 8.794798092957455, -0.4161642018771175, 1.002635508069786)
196 = -0.75787 : (-0.0005053475490047617, 0.0011985002308791141, -0.0018701464802843987, -11.593926533273356, 8.794795183035292, -0.41664760662322325, 1.0026355973002146)
197 = -0.75787 : (-0.0005053246954933417, 0.0011985865793658747, -0.0018714593409622751, -11.593882828853644, 8.794789595152295, -0.41713388879638097, 1.0026355883590712)
198 = -0.75787 : (-0.0005053424334673574, 0.0011986013647587574, -0.001872757162176601, -11.593966312928664, 8.794780064453313, -0.41761487882506676, 1.0026356713329256)
199 = -0.75787 : (-0.0005052941562287048, 0.0011986927559517345, -0.0018741226712273113, -11.593899763501563, 8.794786523379305, -0.4180985528205414, 1.0026356680571598)
200 = -0.75787 : (-0.0005053277610317266, 0.0011987339108821804, -0.0018754493809777734, -11.594030415950808, 8.794759683182766, -0.4185682559775823,

244 = -0.75786 : (-0.0005054097227851309, 0.0011997338001041343, -0.0019130349586925968, -11.594362840389778, 8.79458330279271, -0.4285965426921715, 1.0026365813133231)
245 = -0.75786 : (-0.0005054323801988444, 0.0011997632129254914, -0.0019136612561737498, -11.59439978492254, 8.794578261569857, -0.4287127720860755, 1.002636628678795)
246 = -0.75786 : (-0.0005054309179797816, 0.0011997873144685102, -0.0019143160130683265, -11.594372792192537, 8.794577109298979, -0.42883180783032826, 1.002636608238822)
247 = -0.75786 : (-0.000505430136571133, 0.00119978601618097, -0.0019149424322904094, -11.594417048916096, 8.794575901406986, -0.42894555963687897, 1.002636646943664)
248 = -0.75786 : (-0.0005054152277325248, 0.001199803934005417, -0.001915599407640129, -11.594379965254035, 8.794573366414395, -0.42906182574380375, 1.0026366195772227)
249 = -0.75786 : (-0.0005054112443386666, 0.0011997965954057414, -0.0019162073412963524, -11.59443658302822, 8.794573538884064, -0.42916996490505316, 1.00263

In [6]:
print("-------")
print(outTx)
print(f"Optimizer stop condition: {R.GetOptimizerStopConditionDescription()}")
print(f" Iteration: {R.GetOptimizerIteration()}")
print(f" Metric value: {R.GetMetricValue()}")

sitk.WriteTransform(outTx, "registered_similarity_3D.tfm")

-------
itk::simple::Transform
 Similarity3DTransform (0x7fb14e068d00)
   RTTI typeinfo:   itk::Similarity3DTransform<double>
   Reference Count: 3
   Modified Time: 3152
   Debug: Off
   Object Name: 
   Observers: 
     none
   Matrix: 
     1.00263 0.003863 0.00240839 
     -0.00386543 1.00263 0.0010088 
     -0.00240448 -0.00101807 1.00263 
   Offset: [-11.2282, 9.01899, 0.0637135]
   Center: [14.8988, 2.93955, -173.061]
   Translation: [-11.5945, 8.79454, -0.430839]
   Inverse: 
     0.99736 -0.00384513 -0.00239185 
     0.00384271 0.997362 -0.00101273 
     0.00239574 0.0010035 0.997367 
   Singular: 0
   Versor: [ -0.000505387, 0.00120006, -0.00192703, 0.999997 ]
   Scale = 1.00264

Optimizer stop condition: RegularStepGradientDescentOptimizerv4: Step too small after 271 iterations. Current step (6.10352e-05) is less than minimum step (0.0001).
 Iteration: 272
 Metric value: -0.7578561510144566


In [7]:
if ("SITK_NOSHOW" not in os.environ):
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(fixed)
    resampler.SetInterpolator(sitk.sitkLinear)
    resampler.SetDefaultPixelValue(1)
    resampler.SetTransform(outTx)

    out = resampler.Execute(moving)

    simg1 = sitk.Cast(sitk.RescaleIntensity(fixed), sitk.sitkUInt8)
    simg2 = sitk.Cast(sitk.RescaleIntensity(out), sitk.sitkUInt8)
    cimg = sitk.Compose(simg1, simg2, simg1 // 2. + simg2 // 2.)
    sitk.Show(cimg, "ImageRegistration2 Composition")